In [33]:
import pandas as pd
import glob, os
import time
import datetime
import csv


localdate = datetime.datetime.now()
meter_no = ''
newdate = ''

def normalizeDate(d):
    return '-'.join(((d[15:19]), d[12:14], d[9:11]))

def check_file_writable(fnm):
    print(fnm)
    if os.path.exists(fnm):
        # path exists
        if os.path.isfile(fnm): # is it a file or a dir?
            # also works when file is a link and the target is writable
            return os.access(fnm, os.W_OK)
        else:
            return False # path is a dir, so cannot write as a filehttps://www.amazon.com/gp/f.html?C=1MBFAF2CMECWR&K=I58E44X4ANJT&ami-061bed64019e30a58M=urn:rtn:msg:2021010903002682729c00518542148990b3074c10p0na&R=3S8ALSTPA5VWH&T=C&U=https%3A%2F%2Fconsole.aws.amazon.com%2Fbilling%2Fhome%3Fref_%3Dpe_32559890_475255360%23%2Fbills%3F&H=EQZ2XYBMRCSEU6ZGE8YNZZU1AV4A&ref_=pe_32559890_475255360
    # target does not exist, check perms on parent dir
    pdir = os.path.dirname(fnm)
    if not pdir: pdir = '.'
    # target is creatable if parent dir is writable
    return os.access(pdir, os.W_OK)


# /home/jovyan/work/waterdata/unformatted_data/stream/

# for files in glob.glob("/home/jovyan/work/waterdata/unformatted_data/stream/419051*"):
#    check_file_writable(files)

#for files in glob.glob("/home/jovyan/work/waterdata/unformatted_data/stream/*.csv"):
#    check_file_writable(files)

files = glob.glob("/home/jovyan/work/waterdata/unformatted_data/stream/*.csv")

df1 = pd.concat([pd.read_csv(fp, index_col=False, header=None, skiprows=[0,1,2,3], usecols=[0,1,2,3,4,5], \
    engine='python').assign(meter_no=os.path.basename(fp)) for fp in files])

df1.fillna(value=0,inplace=True)


df1.round({'3' : 2, '5' : 2, '7' : 2})

#initialise variables
mid = None
mn = ''
read_date = ''
sl_read1 = 0.0
ql_read1 = 0
sl_read2 = 0.0
ql_read2 = 0
sl_read3 = 0.0
ql_read3 = 0
comments='Initial load'
creation_date = localdate.strftime('%Y-%m-%d')

fields = []
with open('/home/jovyan/work/waterdata/formatted_data/fmt_all_stream_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile,dialect='excel')

    for i in range(len(df1)):

        read_date = normalizeDate(df1.iloc[i, 0])
        sl_read1 = df1.iloc[i, 1]
        ql_read1 = df1.iloc[i, 2]
        sl_read2 = df1.iloc[i, 3]
        ql_read2 = df1.iloc[i, 4]
        mn = df1.iloc[i, 6]
        mid = i

        fields = [mid,mn[:-4],read_date,sl_read1,ql_read1,sl_read2,ql_read2,sl_read3,ql_read3,comments,creation_date]

        writer.writerow(map(lambda x: x, fields))


